In [1]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.svm import SVR, SVC

from sklearn.metrics import mean_squared_error, f1_score

In [2]:
# Генерируем уникальный seed
my_code = "Suminov"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [3]:
# Читаем данные из файла
example_data = pd.read_csv("datasets/Fish.csv")

In [4]:
example_data.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [5]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(example_data))
print(val_test_size)

32


In [6]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(example_data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

95 32 32


In [7]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
num_columns = ['Weight', 'Length1', 'Length2', 'Length3', 'Height', 'Width']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), num_columns)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['Weight', 'Length1', 'Length2', 'Length3',
                                  'Height', 'Width'])])

In [8]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [9]:
# Устанавливаем названия столбцов
column_names = num_columns + ['Species']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [10]:
sc_train

,Weight,Length1,Length2,Length3,Height,Width,Species
0,0.003952,0.094845,0.089147,0.09058,0.018304,0.015126,Smelt
1,0.316229,0.670103,0.660853,0.664855,0.291237,0.520889,Pike
2,0.422872,0.501031,0.515504,0.576087,0.809955,0.784823,Bream
3,0.372687,0.451546,0.457364,0.514493,0.808118,0.697887,Bream
4,0.623612,0.614433,0.612403,0.628623,0.623042,0.847096,Whitefish
...,...,...,...,...,...,...,...
90,0.040211,0.169072,0.174419,0.175725,0.156991,0.281287,Perch
91,0.404052,0.597938,0.593023,0.59058,0.549984,0.764676,Perch
92,0.148109,0.323711,0.329457,0.384058,0.572992,0.451574,Bream
93,0.001004,0.051546,0.040698,0.050725,0.0,0.0,Smelt


In [11]:
# Задание №1 - анализ метода опорных векторов в задаче регрессии
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR
# kernel : {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'}, default='rbf'
# Только для kernel = 'poly' : degreeint, default=3

In [12]:
# Выбираем 4 числовых переменных, три их них будут предикторами, одна - зависимой переменной
n = 4
labels = random.sample(num_columns, n)

y_label = labels[0]
x_labels = labels[1:]

print(x_labels)
print(y_label)

['Weight', 'Width', 'Length3']
Length1


In [13]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_label]
y_test = sc_test[y_label]
y_val = sc_val[y_label]

In [14]:
x_train

,Weight,Width,Length3
0,0.003952,0.015126,0.09058
1,0.316229,0.520889,0.664855
2,0.422872,0.784823,0.576087
3,0.372687,0.697887,0.514493
4,0.623612,0.847096,0.628623
...,...,...,...
90,0.040211,0.281287,0.175725
91,0.404052,0.764676,0.59058
92,0.148109,0.451574,0.384058
93,0.001004,0.0,0.050725


In [15]:
# Создайте 4 модели с различными ядрами: 'linear', 'poly', 'rbf', 'sigmoid'.
# Решите получившуюся задачу регрессии с помощью созданных моделей и сравните их эффективность.
# При необходимости применяйте параметр регуляризации C : float, default=1.0
# Укажите, какая модель решает задачу лучше других.

r_model_1 = SVR( kernel='linear', C=0.8)
r_model_2 = SVR(kernel='poly', degree=3, C=1.0)
r_model_3 = SVR(kernel='rbf', C=1.0)
r_model_4 = SVR(kernel='sigmoid', C=0.6)

In [16]:
r_models = []
r_models.append(r_model_1)
r_models.append(r_model_2)
r_models.append(r_model_3)
r_models.append(r_model_4)

In [17]:
# Обучаем модели
for model in r_models:
    model.fit(x_train, y_train)

In [18]:
# Оценииваем качество работы моделей на валидационной выборке
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(mse)

0.00417907141962446
0.010600818111692433
0.006886734087134331
5.630402367544336


In [19]:
# Выбираем лучшую модель
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model.get_params()

{'C': 0.8,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.1,
 'gamma': 'scale',
 'kernel': 'linear',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [20]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

0.0045920277071451205


In [21]:
# Задание №2 - анализ метода опорных векторов в задаче классификации
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC
# kernel : {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'}, default='rbf'
# Только для kernel = 'poly' : degreeint, default=3

In [22]:
# Выбираем 2 числовых переменных, которые будут параметрами элементов набора данных
# Метка класса всегда 'Species'
n = 2
x_labels = random.sample(num_columns, n)
y_label = 'Species'

print(x_labels)
print(y_label)

['Height', 'Length2']
Species


In [23]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_label]
y_test = sc_test[y_label]
y_val = sc_val[y_label]

In [24]:
x_train

,Height,Length2
0,0.018304,0.089147
1,0.291237,0.660853
2,0.809955,0.515504
3,0.808118,0.457364
4,0.623042,0.612403
...,...,...
90,0.156991,0.174419
91,0.549984,0.593023
92,0.572992,0.329457
93,0.0,0.040698


In [25]:
# Создайте 4 модели с различными ядрами: 'linear', 'poly', 'rbf', 'sigmoid'.
# Решите получившуюся задачу регрессии с помощью созданных моделей и сравните их эффективность.
# При необходимости применяйте параметр регуляризации C : float, default=1.0
# Укажите, какая модель решает задачу лучше других.
c_model = SVC()
c_model_1 = SVC(kernel='linear', C=0.8)
c_model_2 = SVC(kernel='poly', degree=3, C=1.0)
c_model_3 = SVC(kernel='rbf', C=1.0)
c_model_4 = SVC(kernel='sigmoid', C=0.6)

In [26]:
c_models = []
c_models.append(c_model_1)
c_models.append(c_model_2)
c_models.append(c_model_3)
c_models.append(c_model_4)

In [27]:
#Обучаем модели
for model in c_models:
    model.fit(x_train, y_train)

In [28]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model in c_models:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    f1s.append(f1)
    print(f1)

0.41228991596638653
0.8767241379310344
0.8767241379310344
0.13541666666666669


In [29]:
# Выбираем лучшую модель
i_min = f1s.index(min(f1s))
best_c_model = c_models[i_min]
best_c_model.get_params()

{'C': 0.6,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'sigmoid',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [30]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_c_model.predict(x_test)
f1 = f1_score(y_test, test_pred, average='weighted')
print(f1)

0.26436781609195403
